In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [13]:
from mis_architecture import MIS
from transformers import AutoTokenizer
from tqdm import tqdm
import torch

In [4]:
source_texts = ['I want to leave this room',
               'Hello world, my name is Nick']

paraphrases = ['I want to go out of this room',
               'Hello world, my surname is Petrov']

In [5]:
from torch.utils.data import Dataset, DataLoader
class PairsDatasetInference(Dataset):
    def __init__(self,texts_first, texts_second):
        self.texts_first =texts_first
        self.texts_second = texts_second
    def __len__(self):
        return len(self.texts_first)
    def __getitem__(self, idx):
        return self.texts_first[idx], self.texts_second[idx]

In [10]:
model = MIS.from_pretrained('SkolkovoInstitute/Mutual_Implication_Score')#.cuda()
tokenizer = AutoTokenizer.from_pretrained('SkolkovoInstitute/Mutual_Implication_Score')

In [11]:
dataset_direct = PairsDatasetInference(source_texts, paraphrases)
dataloader_direct = DataLoader(dataset_direct, batch_size = 16)

dataset_reverse= PairsDatasetInference(paraphrases, source_texts)
dataloader_reverse = DataLoader(dataset_reverse, batch_size = 16)

In [15]:
tq = tqdm(zip(dataloader_direct, dataloader_reverse),total = len(dataloader_reverse))

preds = []

for b1,b2 in tq:

    with torch.no_grad():
        tokenized1 = tokenizer(*b1, padding=True, truncation='longest_first', return_tensors="pt")#.to(torch.device('cuda'))
        tokenized2 = tokenizer(*b2, padding=True, truncation='longest_first', return_tensors="pt")#.to(torch.device('cuda'))

        merged_prob = model(tokenized1, tokenized2)

        merged_prob = torch.sigmoid(merged_prob)

    merged_prob = merged_prob.cpu().numpy()
    preds.extend(merged_prob)

preds_fl = [float(e) for e in preds]

100%|██████████| 1/1 [00:00<00:00,  1.89it/s]


In [16]:
preds_fl

[0.9748055338859558, 0.054514043033123016]